<h1 style="text-align: center; border-bottom: 2px solid #ccc; padding-bottom: 10px;">
  Avaliação de Produtos
</h1>

### 1.0 Bibliotecas

In [ ]:
#bibliotecas
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#pré-processamento
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

#modelos
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

#métricas
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

#nuvem de palavras
from wordcloud import WordCloud

### 2.0 Importando dados

In [ ]:
df = pd.read_csv(r"C:\Users\leticia.andradeo\Área de Trabalho\AM\avaliacoes_classificadas.csv", sep=",")
df

In [ ]:
#verificar qtd
df.shape

In [ ]:
#qtd de valores únicos
df.nunique()

In [ ]:
#verificar valores vazios
df.isnull().sum()

In [ ]:
# 5 primeiras linhas
df.head()

### 3.0 Exploração dos dados

#### 3.1 Tipo de produto

In [ ]:
df['Consulta_de_Busca'].unique()

In [ ]:
#valores únicos
df['Consulta_de_Busca'].nunique()

#### 3.2 Marcas

In [ ]:
#Retire o comentário para vizualizar. São muitas obs. 
#df['Titulo_do_Produto'].unique()

In [ ]:
#valores únicos
df['Titulo_do_Produto'].nunique()

#### 3.3 Distribuição da quantidade de palavras nas avaliações

In [ ]:
# Tamanho dos textos
df['tamanho_texto'] = df['Avaliacao'].apply(lambda x: len(str(x).split()))
sns.histplot(df['tamanho_texto'], bins=5)
plt.title('Distribuição do tamanho das avaliações')
plt.xlabel('Tamanho das avaliações') # Eixo x
plt.ylabel('Frequência') # Eixo y

In [ ]:
# Criar o boxplot
plt.figure(figsize=(8, 6)) # Ajuste do tamanho da figura
plt.boxplot(df['tamanho_texto'], vert=False)  # Se vert=False, o boxplot será horizontal
plt.xlabel('Tamanho das avaliações') # Eixo x
plt.title('Distribuição do tamanho dos dados')  # Ttítulo do gráfico
print(df['tamanho_texto'].describe()) # resumo estatístico
plt.show()  # Exibir o gráfico

### 4.0 Classificador por contagem de palavras

#### 4.1 Pré - processamento

In [ ]:
lemmatizer = WordNetLemmatizer() #lematização
stop_words = set(stopwords.words("portuguese")) # stopword

In [ ]:
def preprocessamento(texto):
    text = texto.lower() #ransforma todo o texto em letras minúsculas,
    text = re.sub(r"[^\w\s]", "", texto) # Usa expressão regular para eliminar qualquer caractere que não seja uma letra, número ou espaço em branco.
    tokens = nltk.word_tokenize(texto) # Tokeniza o texto, ou seja, divide o texto em palavras (tokens) usando a função word_tokenize do NLTK.
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words] #Remove stopwords #Aplica lematização, ou seja, reduz as palavras à sua forma base (por exemplo, "correram" → "correr").
    return ' '.join(tokens)                                                   

In [ ]:
#Aplicação da função
df['Avaliacao_Tratada'] = df['Avaliacao'].apply(preprocessamento)

#### 4.2 Palavras positivas

In [ ]:
# Base de palavras positivas
base_positiva = [
    "ótimo", "excelente", "maravilhoso", "perfeito", "bom", "boa", "funciona", "rápido", "eficiente", 
    "moderno", "bonito", "design", "resistente", "durável", "bateria", "leve", "prático", 
    "compacto", "potente", "intuitivo", "fácil", "responsivo", "incrível", "suave", "estável",
    "seguro", "custo-benefício", "recomendo", "top", "nítido", "vale a pena", "memória boa",
    "ótima câmera", "carregamento rápido", "som perfeito", "imagem clara", "fluido", "qualidade",
    "gostei", "amei", "adorei", "recomendo", "encantou", "atendeu", 
    "funcionou", "surpreendeu", "satisfeito", "agradou", "valeu", "curti", "comprem sem medo","não trava",

    # Adições por categoria de produto
    "tela vibrante", "imagem nítida", "cores vivas",  # TV, tablet, smartphone
    "som potente", "áudio limpo", "graves fortes",    # caixas de som, fones de ouvido
    "boa conectividade", "bluetooth estável", "pareamento fácil",  # fones, smartwatch
    "boa autonomia", "carregamento eficiente",        # drone, smartwatch
    "resposta rápida", "baixa latência",              # mouse, teclado, console
    "corte preciso", "lâmina afiada",                 # barbeador
    "impressão de qualidade", "rendimento bom",       # impressora
    "armazenamento rápido", "transferência veloz",    # SSD
    "esquenta rápido", "faz café delicioso",          # cafeteira
    "muito silencioso", "funcionamento silencioso",   # ventilador, aspirador
    "boa refrigeração", "resfria rápido",             # ar-condicionado
    "costura perfeita", "acabamento impecável",       # roupas, vestido, camisa
    "material de qualidade", "tecido confortável",    # vestuário
    "encaixe perfeito", "confortável no pé",          # tênis, sapato
    "ajuste anatômico", "apoio ergonômico",           # cadeira
    "cheiro agradável", "fixação prolongada",         # perfume
    "joia linda", "brilho intenso",                   # joias
    "tablet rápido", "ideal para leitura",            # tablet
]


# Criar DataFrame 
df_positivas = pd.DataFrame(base_positiva)

#### 4.3 Palavras negativas

In [ ]:
# Base de palavras negativas
base_negativas = [
    "ruim", "péssimo", "fraco", "horrível", "lento", "problemático", "decepcionante", "trava",
    "não funciona", "quebra", "frágil", "esquenta", "bugado", "descarrega", "erro", 
    "não recomendo", "baixo desempenho", "câmera ruim", "som baixo", "insatisfatório",
    "não gostei", "odiei", "detestei", "não recomendo", "decepcionou", 
    "quebrou", "esquentou", "travou", "falhou", "pifou", "insatisfeito",

    # Adições por categoria
    "imagem borrada", "cores apagadas", "tela escura","lento", "esquenta","frágil", "fraco",      # TV, celular
    "áudio chiado", "sem graves", "microfone ruim",         # fones, caixas de som
    "pareamento difícil", "bluetooth falha",                # fones, smartwatch
    "autonomia ruim", "carrega devagar",                    # smartwatch, drones
    "resposta lenta", "input delay",                        # mouse, controle, teclado
    "corte irregular", "lâmina cega",                       # barbeador
    "papel enrosca", "mancha impressão",                    # impressora
    "memória lenta", "queda de desempenho",                 # SSD
    "faz muito barulho", "ruído alto",                      # aspirador, ventilador
    "não refrigera", "ar fraco",                            # ar-condicionado
    "tecido áspero", "acabamento ruim",                     # roupas
    "desconfortável", "machuca o pé",                       # sapato, tênis
    "cadeira desconfortável", "apoio frágil",               # cadeira escritório
    "cheiro fraco", "duração ruim",                         # perfume
    "joia opaca", "fecho frágil",                           # joias
    "tablet travando", "não roda apps",                     # tablet
]


# Criar DataFrame
df_negativas = pd.DataFrame(base_negativas)

#### 4.4 Aplicação do classificador 

In [ ]:
def classificador_de_sentimento(texto, pos_set, neg_set):
    tokens = texto.lower().split()  # coloca tudo em minúsculas e cria os tokens
    qtd_positivo = sum(1 for palavra in tokens if palavra in pos_set.values)
    qtd_negativo = sum(1 for palavra in tokens if palavra in neg_set.values)

    if qtd_positivo > qtd_negativo:
        return "positivo"
    elif qtd_negativo > qtd_positivo:
        return "negativo"
    else:
        return "neutro"

In [ ]:
# Aplicar classificador
df['Sentimento'] = df['Avaliacao_Tratada'].apply(lambda x: classificador_de_sentimento(x, df_positivas, df_negativas))

In [ ]:
#verificar saída
df['Sentimento'].unique()

In [ ]:
#5 primeiras linhas
df.head()

#### 4.5 Resultado do classificador

In [ ]:
# Contagem absoluta por sentimento e produto
resultado_produto_class = pd.DataFrame(df.groupby('Consulta_de_Busca')['Sentimento'].value_counts().unstack(fill_value=0))

# Muda o nome do índice das colunas
resultado_produto_class.rename_axis(columns='Índice', inplace=True)

# Calcula o total por linha (para percentuais)
total_por_linha_pro_class = resultado_produto_class.sum(axis=1)

# Cria novas colunas com o percentual
resultado_produto_class['negativo_%'] = (resultado_produto_class['negativo'] / total_por_linha_pro_class * 100).round(2)
resultado_produto_class['neutro_%']   = (resultado_produto_class['neutro'] / total_por_linha_pro_class * 100).round(2)
resultado_produto_class['positivo_%'] = (resultado_produto_class['positivo'] / total_por_linha_pro_class * 100).round(2)

# Visualiza
resultado_produto_class.reset_index(inplace=True)
resultado_produto_class

#exportar para excel
resultado_produto_class.to_excel('resultado_produto_class.xlsx', index=True)

In [ ]:
# Contagem absoluta por sentimento e produto e marca
resultado_marca_class = pd.DataFrame(df.groupby(['Consulta_de_Busca','Titulo_do_Produto'])['Sentimento'].value_counts().unstack(fill_value=0))

# Muda o nome do índice das colunas
resultado_marca_class.rename_axis(columns='Índice', inplace=True)

# Calcula o total por linha (para percentuais)
total_por_linha_marca_class = resultado_marca_class.sum(axis=1)

# Cria novas colunas com o percentual
resultado_marca_class['negativo_%'] = (resultado_marca_class['negativo'] / total_por_linha_marca_class * 100).round(2)
resultado_marca_class['neutro_%']   = (resultado_marca_class['neutro'] / total_por_linha_marca_class * 100).round(2)
resultado_marca_class['positivo_%'] = (resultado_marca_class['positivo'] / total_por_linha_marca_class * 100).round(2)

# Visualiza
resultado_marca_class.reset_index(inplace=True)
resultado_marca_class

#exportar para excel
resultado_marca_class.to_excel('resultado_marca_class.xlsx', index=True)

#### 4.6 Nuvem de palavras

In [ ]:
# Nuvem de palavras
texto_positivo = " ".join(df[df["Sentimento"] == 'positivo']["Avaliacao_Tratada"])
texto_negativo = " ".join(df[df["Sentimento"] == 'negativo']["Avaliacao_Tratada"])

In [ ]:
WordCloud().generate(texto_positivo).to_image()

In [ ]:
WordCloud().generate(texto_negativo).to_image()

### 5.0 Modelo MultinomialNB com a técnica CountVectorizer

#### 5.1 Base de dados com target - classificada

In [ ]:
#vamos utilizar apenas os casos classificados
df_com_target = df_com_target = df[df['target'].notna()]

In [ ]:
#qtd linhas e colunas
df_com_target.shape

In [ ]:
df_com_target.head()

#### 5.2 Vetorização das palavras com CountVectorizer() e treinamento do modelo MultinomialNB

In [ ]:
#Vetorização BoW
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_com_target['Avaliacao_Tratada'])
y = df_com_target['target']

#Split treino e teste 70/30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#Treinamento do modelo Naive Bayes
modelo_nb = MultinomialNB()
modelo_nb.fit(X_train, y_train)

#### 5.3 Avaliação do modelo MultinomialNB

In [ ]:
y_pred_nb = modelo_nb.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred_nb))
mc_nb = confusion_matrix(y_test, y_pred_nb) # matriz de confusão Naive Bayes
mc_disp_nb = ConfusionMatrixDisplay(confusion_matrix=mc_nb)
mc_disp_nb.plot()

#### 5.4 Resultado do modelo MultinomialNB

In [ ]:
# Vetorizando
Avaliacao_vet_nb = vectorizer.transform(df['Avaliacao_Tratada']) # vectorizer = CountVectorizer()

# Agora você pode prever:
df['Previsao_NB'] = modelo_nb.predict(Avaliacao_vet_nb)

#transformar
df['Previsao_NB'] = df['Previsao_NB'].map({0: 'negativo', 1: 'positivo'})

In [ ]:
df.head()

In [ ]:
# Contagem absoluta por sentimento e produto
resultado_produto_nb = pd.DataFrame(df.groupby('Consulta_de_Busca')['Previsao_NB'].value_counts().unstack(fill_value=0))

# Muda o nome do índice das colunas
resultado_produto_nb.rename_axis(columns='Índice', inplace=True)

# Calcula o total por linha (para percentuais)
total_por_linha_nb = resultado_produto_nb.sum(axis=1)

# Cria novas colunas com o percentual
resultado_produto_nb['negativo_%'] = (resultado_produto_nb['negativo'] / total_por_linha_nb * 100).round(2)
resultado_produto_nb['positivo_%'] = (resultado_produto_nb['positivo'] / total_por_linha_nb * 100).round(2)

# Visualiza
resultado_produto_nb.reset_index(inplace=True)
resultado_produto_nb

#exportar para excel
resultado_produto_nb.to_excel('resultado_produto_nb.xlsx', index=True)

In [ ]:
# Contagem absoluta por sentimento e produto e marca
resultado_marca_nb = pd.DataFrame(df.groupby(['Consulta_de_Busca','Titulo_do_Produto'])['Previsao_NB'].value_counts().unstack(fill_value=0))

# Muda o nome do índice das colunas
resultado_marca_nb.rename_axis(columns='Índice', inplace=True)

# Calcula o total por linha (para percentuais)
total_por_linha_marca_nb = resultado_marca_nb.sum(axis=1)

# Cria novas colunas com o percentual
resultado_marca_nb['negativo_%'] = (resultado_marca_nb['negativo'] / total_por_linha_marca_nb * 100).round(2)
resultado_marca_nb['positivo_%'] = (resultado_marca_nb['positivo'] / total_por_linha_marca_nb * 100).round(2)

# Visualiza
resultado_marca_nb.reset_index(inplace=True)
resultado_marca_nb

#exportar para excel
resultado_marca_nb.to_excel('resultado_marca_nb.xlsx', index=True)

#### 5.5 Nuvem de palavras

In [ ]:
# Nuvem de palavras
texto_positivo = " ".join(df[df["Previsao_NB"] == 'positivo']["Avaliacao_Tratada"])

In [ ]:
# Nuvem de palavras
texto_negativo = " ".join(df[df["Previsao_NB"] == 'negativo']["Avaliacao_Tratada"])

In [ ]:
WordCloud().generate(texto_positivo).to_image()

In [ ]:
WordCloud().generate(texto_negativo).to_image()

### 6.0 Modelo com LogisticRegression com TfidfVectorizer

#### 6.1 Vetorizaçao com TfidfVectorizere treinamento do modelo LogisticRegression

In [ ]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df_com_target["Avaliacao_Tratada"])
y = df_com_target["target"]

#Split treino e teste 70/30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#Treinando o modelo LogisticRegression()
modelo_lr = LogisticRegression()
modelo_lr.fit(X_train, y_train)

#### 6.2 Avaliação do modelo LogisticRegression()

In [ ]:
y_pred_lr = modelo_lr.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred_lr))
mc_lr = confusion_matrix(y_test, y_pred_lr)
mc_disp_lr = ConfusionMatrixDisplay(confusion_matrix=mc_lr)
mc_disp_lr.plot()

#### 6.3 Resultado do modelo LogisticRegression()

In [ ]:
# Vetorizando
Avaliacao_vet_lr = tfidf.transform(df['Avaliacao_Tratada'])  # tfidf = TfidfVectorizer()

# Agora você pode prever:
df['Previsao_LR'] = modelo_lr.predict(Avaliacao_vet_lr)

#transformar
df['Previsao_LR'] = df['Previsao_LR'].map({0: 'negativo', 1: 'positivo'})

In [ ]:
# Contagem absoluta por sentimento e produto
resultado_produto_lr = pd.DataFrame(df.groupby('Consulta_de_Busca')['Previsao_LR'].value_counts().unstack(fill_value=0))

# Muda o nome do índice das colunas
resultado_produto_lr.rename_axis(columns='Índice', inplace=True)

# Calcula o total por linha (para percentuais)
total_por_linha_lr = resultado_produto_lr.sum(axis=1)

# Cria novas colunas com o percentual
resultado_produto_lr['negativo_%'] = (resultado_produto_lr['negativo'] / total_por_linha_lr * 100).round(2)
resultado_produto_lr['positivo_%'] = (resultado_produto_lr['positivo'] / total_por_linha_lr * 100).round(2)

# Visualiza
resultado_produto_lr.reset_index(inplace=True)
resultado_produto_lr

#exportar para excel
resultado_produto_lr.to_excel('resultado_produto_lr.xlsx', index=True)

In [ ]:
# Contagem absoluta por sentimento e produto e marca
resultado_marca_lr = pd.DataFrame(df.groupby(['Consulta_de_Busca','Titulo_do_Produto'])['Previsao_LR'].value_counts().unstack(fill_value=0))

# Muda o nome do índice das colunas
resultado_marca_lr.rename_axis(columns='Índice', inplace=True)

# Calcula o total por linha (para percentuais)
total_por_linha_marca_lr = resultado_marca_lr.sum(axis=1)

# Cria novas colunas com o percentual
resultado_marca_lr['negativo_%'] = (resultado_marca_lr['negativo'] / total_por_linha_marca_lr * 100).round(2)
resultado_marca_lr['positivo_%'] = (resultado_marca_lr['positivo'] / total_por_linha_marca_lr * 100).round(2)

# Visualiza
resultado_marca_lr.reset_index(inplace=True)
resultado_marca_lr

#exportar para excel
resultado_marca_lr.to_excel('resultado_marca_lr.xlsx', index=True)

#### 6.4 Nuvem de palavras

In [ ]:
texto_positivo = " ".join(df[df["Previsao_LR"] == 'positivo']["Avaliacao_Tratada"])

In [ ]:
texto_negativo = " ".join(df[df["Previsao_LR"] == 'negativo']["Avaliacao_Tratada"])

In [ ]:
WordCloud().generate(texto_positivo).to_image()

In [ ]:
WordCloud().generate(texto_positivo).to_image()

### 7.0 Comparação das abordagens

In [ ]:
# Vetorização com CountVectorizer()
# classificador Naive Bayes
print(classification_report(y_test, y_pred_nb))

In [ ]:
#Vetorização com TfidfVectorizer()
#Classificador Regressão Logística
print(classification_report(y_test, y_pred_lr))